# Team 88: Aggregation
## Airport Traffic Data

From the initial review of this dataset, we identified variables of interest, useful supplementary variables from other datasets, and any needed cleaning was completed. Data from the source is only available through June 2020 though our covered timeframe will end on Dec 19 2019 for datasets (if available). Now with the variables and data set, we can aggregate by our team agreed upon measures:
- Time (date)
- Location (Longitude and Latitude, zip codes where available)

For this dataset, we have to find a way to extract dates because they are not readily provided, and we have the origin airport lat/lon values. We don't need to include the destination ones because airport arrivals are going to be more generally analyzed in the bay area. We can't know for sure whether people are transiting or staying in the bay area, but can at least see where they are flying in from to inform final visuals.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#modules
import pandas as pd
import numpy as np

!pip install datetime-quarter  #uncomment and run if needed
from datequarter import DateQuarter

Reminder of the fields in the datasets. All of these descriptions come from the data source.
- `ITIN_ID` = Itinerary ID
- `ORIGIN_AIRPORT_ID` = Origin Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
- `ORIGIN` = Origin Airport Code
- `DEST_AIRPORT_ID` = Destination Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
- `DEST` = Destination Airport Code
- `PASSENGERS` = Number of Passengers for the itinerary
- `YEAR` = Year
- `QUARTER` = Quarter (1-4)
- `ORIGIN_COUNTRY` = Origin Airport, Country Code
- `ROUNDTRIP` = Round Trip Indicator (1=Yes)
- `ITIN_FARE` = Itinerary Fare Per Person
- `ORIGIN_CITY_NAME` = collected as `AirportCityName`: Airport City Name with either U.S. State or Country
- `DEST_CITY_NAME` = collected as `AirportCityName`: Airport City Name with either U.S. State or Country

In [ ]:
#import data
path = '/content/drive/MyDrive/2020-Move/Learning/DS4A-correlation-one/DS4A Project/repo/'
all_travel = pd.read_csv(path+'data/all_travel.csv').drop(['Unnamed: 0', 'ORIGIN', 'DEST'], axis=1)
print(all_travel.shape,'\n')
all_travel.head(2)

(4288742, 9) 



,ITIN_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,PASSENGERS,YEAR,QUARTER,ORIGIN_COUNTRY,ROUNDTRIP,ITIN_FARE
0,20111156458,11278,14771,1.0,2011,1,US,0.0,2112.0
1,20111156497,11618,14831,1.0,2011,1,US,0.0,394.0


By far, most people fly in from LAX (Los Angeles International Airport) followed by a few west coast airports and some major hubs including New York (JFK) and Boston (BOS). Given that airport codes are not common knowledge, a supplemental file with the city names for each airport code is included and used to update the plot above for top 25 origin airports for other uses going forward.

The [Market Coordinate](https://www.transtats.bts.gov/Tables.asp?DB_ID=595&DB_Name=Aviation%20Support%20Tables) table download (`341379231_T_MASTER_CORD.csv`) that is also provided by the Bureau of Transportation Statistics as with [all our data](https://www.transtats.bts.gov/Tables.asp?DB_ID=125&DB_Name=Airline%20Origin%20and%20Destination%20Survey%20%28DB1B%29&DB_Short_Name=Origin%20and%20Destination%20Survey), includes information about airport codes, IDs, and location, along with any historical identifying information about the airports that appear on the itineraries downloaded.

In [ ]:
#get city/location data for merging with
airport_data = pd.read_csv(path+'data/341379231_T_MASTER_CORD.csv')
airport_data.shape

(18102, 14)

In [ ]:
airport_data.head(2)

,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_COUNTRY_NAME,AIRPORT_STATE_NAME,AIRPORT_STATE_FIPS,DISPLAY_CITY_MARKET_NAME_FULL,LAT_DEGREES,LATITUDE,LON_DEGREES,LONGITUDE,AIRPORT_IS_LATEST,Unnamed: 13
0,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",United States,Alaska,2.0,"Afognak Lake, AK",58.0,58.109444,152.0,-152.906667,1,NaN
1,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",United States,Alaska,2.0,"Granite Mountain, AK",65.0,65.548056,161.0,-161.071667,1,NaN


We established during exploration that the parameter that will be used to connect these two datasets is the airport ID = a five digit value that is unique to the airport as the three alphanumeric digit codes tend to be reassigned or retired.

In [ ]:
#create temporary columns for merging// there are only four destinations, only adding origin lat/lon
airport_data['ORIGIN_AIRPORT_ID'] = airport_data['AIRPORT_ID']
airport_data['ORIGIN_CITY_NAME'] = airport_data['DISPLAY_AIRPORT_CITY_NAME_FULL']
airport_data['DEST_AIRPORT_ID'] = airport_data['AIRPORT_ID']
airport_data['DEST_CITY_NAME'] = airport_data['DISPLAY_AIRPORT_CITY_NAME_FULL']
airport_data['ORIGIN_LONGITUDE'] = airport_data['LONGITUDE']
airport_data['ORIGIN_LATITUDE'] = airport_data['LATITUDE']
airport_data.iloc[:, -8:].head(2)

,AIRPORT_IS_LATEST,Unnamed: 13,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,ORIGIN_LONGITUDE,ORIGIN_LATITUDE
0,1,NaN,10001,"Afognak Lake, AK",10001,"Afognak Lake, AK",-152.906667,58.109444
1,1,NaN,10003,"Granite Mountain, AK",10003,"Granite Mountain, AK",-161.071667,65.548056


In [ ]:
#dropping unused/useless columns
del airport_data['Unnamed: 13']
airport_data.head(2)

,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_COUNTRY_NAME,AIRPORT_STATE_NAME,AIRPORT_STATE_FIPS,DISPLAY_CITY_MARKET_NAME_FULL,LAT_DEGREES,LATITUDE,LON_DEGREES,LONGITUDE,AIRPORT_IS_LATEST,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,ORIGIN_LONGITUDE,ORIGIN_LATITUDE
0,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",United States,Alaska,2.0,"Afognak Lake, AK",58.0,58.109444,152.0,-152.906667,1,10001,"Afognak Lake, AK",10001,"Afognak Lake, AK",-152.906667,58.109444
1,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",United States,Alaska,2.0,"Granite Mountain, AK",65.0,65.548056,161.0,-161.071667,1,10003,"Granite Mountain, AK",10003,"Granite Mountain, AK",-161.071667,65.548056


In [ ]:
#saving destinations (there's only four) for later
destinations = all_travel['DEST_AIRPORT_ID'].unique()
dest_cities = airport_data[airport_data['AIRPORT_ID'].isin(destinations)][['AIRPORT_ID', 'DISPLAY_AIRPORT_CITY_NAME_FULL']].drop_duplicates()
dest_cities

,AIRPORT_ID,DISPLAY_AIRPORT_CITY_NAME_FULL
10596,13796,"Oakland, CA"
13121,14771,"San Francisco, CA"
13278,14831,"San Jose, CA"
13858,15023,"Santa Rosa, CA"


After filtering the dataset to only include bay area inbound flights and aggregating all periods, we decided to use December 19th as the cutoff for our data periods. There are no identifiable dates from this data, but the `ITIN_ID` field looks like it contains the date so we try to extract that to get this information. Some things to consider:
- Q1 - Q3: looks like the month is only recorded as a single digit from 1-9 (as opposed to 01-09)
- Need to check dates as some are recorded as above for the 1st to the 9th of any month

In [ ]:
#method to extract portion of ITIN_ID with date and convert to datetime object
def get_date(itin_id, quarter):
  date_str = str(itin_id)
  if quarter in [1, 2, 3]:
    return date_str[:4] + '-' + date_str[4:5] + '-' + date_str[5:7]
  else:
    return date_str[:4] + '-' + date_str[4:6] + '-' + date_str[6:8]

In [ ]:
#trying this with a small sample
trial = all_travel[all_travel['QUARTER']==4][['ITIN_ID', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'QUARTER']]

In [ ]:
func = lambda x, y: get_date(x, y)
trial = trial[:50]
trial['date'] = trial[['ITIN_ID', 'QUARTER']].apply(lambda x: func(*x), axis=1)
trial.head()

,ITIN_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,QUARTER,date
3099794,20104210674,12264,14831,4,2010-42-10
3099795,20104211382,11298,14771,4,2010-42-11
3099796,20104211410,11298,14831,4,2010-42-11
3099797,20104318616,10599,14831,4,2010-43-18
3099798,20104392374,11298,14831,4,2010-43-92


Looks like the hunch about the date is wrong. 
- The first four digits are the year, 
- the fifth one is the quarter number
- No idea what the last six or seven digits represent based on the information provided in the source data page.

In [ ]:
#Q1 check (the last digits after year and quarter)
for quarter in range(1,5):
  vals = all_travel[all_travel['QUARTER']==quarter]['ITIN_ID'].apply(lambda x: str(x)[5:])
  print('\nQ{} \nunique after first 5 digits (year, quarter)'.format(quarter), len(vals.unique()))
  print('total line items', len(vals))
  print('any divisibles in quarter {}?'.format(quarter), len(vals)/len(vals.unique()))


Q1 
unique after first 5 digits (year, quarter) 831193
total line items 942389
any divisibles in quarter 1? 1.133778797463405

Q2 
unique after first 5 digits (year, quarter) 1015846
total line items 1100400
any divisibles in quarter 2? 1.0832350572823046

Q3 
unique after first 5 digits (year, quarter) 957551
total line items 1057005
any divisibles in quarter 3? 1.1038628751888933

Q4 
unique after first 5 digits (year, quarter) 1042976
total line items 1188948
any divisibles in quarter 4? 1.139957199398644


In [ ]:
#Q2 itin id length check
for i in range(1,5):
  print('Q{}'.format(i), all_travel[all_travel['QUARTER']==i]['ITIN_ID'].apply(lambda x: len(str(x))).unique())

Q1 [11 12]
Q2 [11 12 10]
Q3 [11 12]
Q4 [11 12]


For some reason, there are items with length 10. Doesn't matter, still no definition of those other numbers, so there is some investigation to see what these values for each quarter represent.

*Update:* Not a lot of context provided from the source as to what the numbers after year and quarter represent beyond being unique identifiers for that initerary. For now, we aggregate everything by using year and quarter granularity for analysis and visualizations.

In [ ]:
all_travel.head(2)

,ITIN_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,PASSENGERS,YEAR,QUARTER,ORIGIN_COUNTRY,ROUNDTRIP,ITIN_FARE
0,20111156458,11278,14771,1.0,2011,1,US,0.0,2112.0
1,20111156497,11618,14831,1.0,2011,1,US,0.0,394.0


In [ ]:
#create quarter-year datetime object to use as aggregator
func_dt = lambda x, y: DateQuarter(x, y)
all_travel['QQ-YYYY'] = all_travel[['YEAR', 'QUARTER']].apply(lambda x: func_dt(*x), axis=1)

In [ ]:
all_travel = all_travel.drop(['YEAR', 'QUARTER'], axis=1)
all_travel.head(2)

,ITIN_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,PASSENGERS,ORIGIN_COUNTRY,ROUNDTRIP,ITIN_FARE,QQ-YYYY
0,20111156458,11278,14771,1.0,US,0.0,2112.0,Q1 of 2011
1,20111156497,11618,14831,1.0,US,0.0,394.0,Q1 of 2011


Because `ITIN_ID`, `ORIGIN_AIRPORT_ID`, `DEST_AIRPORT_ID` and `ROUNDTRIP` are all categorical variables, it would be nice to make sure they are also not aggregated by converting them to strings before aggregating anything.

**The Numerical Fields**  
Recall that `ITIN_FARE` is a per passenger representation of the cost of airfare and `PASSENGERS` is the number of people associated with an itinerary booking

In [ ]:
print('Confirming data types:')
for column in ['ITIN_ID', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'ROUNDTRIP']:
  all_travel[column] = all_travel[column].astype(str)
  print(column+':', type(all_travel[column][0]))

Confirming data types:
ITIN_ID: <class 'str'>
ORIGIN_AIRPORT_ID: <class 'str'>
DEST_AIRPORT_ID: <class 'str'>
ROUNDTRIP: <class 'str'>


In [ ]:
#aggregating passengers by sum
passenger_agg = all_travel.groupby(by=['QQ-YYYY', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']).sum().reset_index().drop(['ITIN_FARE'], axis=1)
print(passenger_agg.shape)
passenger_agg.head(2)

(33564, 4)


,QQ-YYYY,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,PASSENGERS
0,Q4 of 2010,10135,13796,1.0
1,Q4 of 2010,10135,14771,59.0


In [ ]:
#aggregating itinerary fare by main measures (as this is a per person amount for everyone on the itinerary)
measure_dfs = []
aggregators = ['QQ-YYYY', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']
measure_dfs.append(('MEDIAN', all_travel.groupby(by=aggregators).median().reset_index().drop(['PASSENGERS'], axis=1)))
measure_dfs.append(('AVG', all_travel.groupby(by=aggregators).mean().reset_index().drop(['PASSENGERS'], axis=1)))
measure_dfs.append(('MIN', all_travel.groupby(by=aggregators).min().reset_index().drop(['PASSENGERS', 'ITIN_ID',	'ORIGIN_COUNTRY', 'ROUNDTRIP'], axis=1)))
measure_dfs.append(('MAX', all_travel.groupby(by=aggregators).max().reset_index().drop(['PASSENGERS', 'ITIN_ID',	'ORIGIN_COUNTRY', 'ROUNDTRIP'], axis=1)))
for i, df in enumerate(measure_dfs):
  measure_dfs[i] = df[1].rename(columns={'ITIN_FARE': df[0]+'_ITIN_FARE'})
  print(df[0].title(),'df:', df[1].shape)

#merge fare ones
fare_agg = pd.merge(pd.merge(pd.merge(measure_dfs[0], measure_dfs[1], on=aggregators), measure_dfs[2], on=aggregators), measure_dfs[3], on=aggregators)
print('all fare merge', fare_agg.shape)
fare_agg.head(2)

Median df: (33564, 4)
Avg df: (33564, 4)
Min df: (33564, 4)
Max df: (33564, 4)
all fare merge (33564, 7)


,QQ-YYYY,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,MEDIAN_ITIN_FARE,AVG_ITIN_FARE,MIN_ITIN_FARE,MAX_ITIN_FARE
0,Q4 of 2010,10135,13796,10.0,10.000000,10.0,10.0
1,Q4 of 2010,10135,14771,466.0,457.350877,10.0,1059.0


Before merging the market coordinates data that gives us more location information about the airports, we need to make sure there are no duplicates in that data e.g. the same location represented twice because of spelling differences (encountered during data cleaning and exploration stages).

In [ ]:
print(airport_data.shape)
airport_data.head(2)

(6548, 19)


,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_COUNTRY_NAME,AIRPORT_STATE_NAME,AIRPORT_STATE_FIPS,DISPLAY_CITY_MARKET_NAME_FULL,LAT_DEGREES,LATITUDE,LON_DEGREES,LONGITUDE,AIRPORT_IS_LATEST,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,ORIGIN_LONGITUDE,ORIGIN_LATITUDE
0,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",United States,Alaska,2.0,"Afognak Lake, AK",58.0,58.109444,152.0,-152.906667,1,10001,"Afognak Lake, AK",10001,"Afognak Lake, AK",-152.906667,58.109444
1,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",United States,Alaska,2.0,"Granite Mountain, AK",65.0,65.548056,161.0,-161.071667,1,10003,"Granite Mountain, AK",10003,"Granite Mountain, AK",-161.071667,65.548056


In [ ]:
airport_data = airport_data.drop_duplicates(subset=['AIRPORT_ID'])
airport_data.shape

(6548, 19)

In [ ]:
#merge aggregates into one df
travel_df = pd.merge(passenger_agg, fare_agg, on=['QQ-YYYY','ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID'])
print('aggregate merge', travel_df.shape)

#change airport ID to int again so it can be used to merge
travel_df['ORIGIN_AIRPORT_ID'] = travel_df['ORIGIN_AIRPORT_ID'].astype(int)
travel_df['DEST_AIRPORT_ID'] = travel_df['DEST_AIRPORT_ID'].astype(int)
#merge origin location values from market coordinates dataframe
travel_df = travel_df.merge(airport_data[['ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME']],how='left', on=['ORIGIN_AIRPORT_ID'])
travel_df = travel_df.merge(airport_data[['DEST_AIRPORT_ID', 'DEST_CITY_NAME']],how='left', on=['DEST_AIRPORT_ID'])
print('complete merge', travel_df.shape, '\n')
travel_df.iloc[:5:,-8:]

aggregate merge (33564, 8)
complete merge (33564, 10) 



,DEST_AIRPORT_ID,PASSENGERS,MEDIAN_ITIN_FARE,AVG_ITIN_FARE,MIN_ITIN_FARE,MAX_ITIN_FARE,ORIGIN_CITY_NAME,DEST_CITY_NAME
0,13796,1.0,10.0,10.000000,10.0,10.0,"Allentown/Bethlehem/Easton, PA","Oakland, CA"
1,14771,59.0,466.0,457.350877,10.0,1059.0,"Allentown/Bethlehem/Easton, PA","San Francisco, CA"
2,14831,7.0,536.0,573.285714,321.0,778.0,"Allentown/Bethlehem/Easton, PA","San Jose, CA"
3,13796,1.0,609.0,609.000000,609.0,609.0,"Abilene, TX","Oakland, CA"
4,14771,5.0,660.0,523.800000,10.0,784.0,"Abilene, TX","San Francisco, CA"


In [ ]:
travel_df = travel_df.drop(['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID'], axis=1)
travel_df.columns

Index(['QQ-YYYY', 'PASSENGERS', 'MEDIAN_ITIN_FARE', 'AVG_ITIN_FARE',
       'MIN_ITIN_FARE', 'MAX_ITIN_FARE', 'ORIGIN_CITY_NAME', 'DEST_CITY_NAME'],
      dtype='object')

Now that we have aggregated the fields we want to merge with everything else, we can export this file to be stored with all others.

- `QQ-YYYY` = Quarter and Year
- `PASSENGERS` = Number of Passengers for the quarter based on departure and destination
- `MEDIAN_ITIN_FARE` = Median Itinerary Fare Per Person on the flight route for that quarter
- `AVG_ITIN_FARE` = Average Itinerary Fare Per Person on the flight route for that quarter
- `MIN_ITIN_FARE` = Minimum Itinerary Fare Per Person on the flight route for that quarter
- `MAX_ITIN_FARE` = Maximum Itinerary Fare Per Person on the flight route for that quarter
- `ORIGIN_CITY_NAME` = collected as `AirportCityName`: Airport City Name with either U.S. State or Country
- `DEST_CITY_NAME` = collected as `AirportCityName`: Airport City Name with either U.S. State or Country

In [ ]:
travel_df.to_csv(path+'data/airport_data_aggregates.csv')